In [1]:
import openai

In [6]:
from openai import OpenAI
import json

In [4]:
client = OpenAI(api_key="sk-OPEN_AI_API_KEY")

# Image-to-text pre-trained model

In [5]:
'''
This code uses the OpenAI client object to make a chat completion request using the GPT-4 Turbo model.
It sends a user message asking "What’s in this image?" along with an image URL. The response from the
model is stored in the response variable. The code then prints the assistant's response, which can be 
accessed using response.choices[0].
''' 
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
    "https://cdn.pixabay.com/photo/2017/08/25/18/48/watercolor-2681039_1280.jpg",
    "https://cdn.pixabay.com/photo/2017/03/12/13/41/colorful-2137080_1280.jpg",
    "https://cdn.pixabay.com/photo/2018/03/30/15/11/deer-3275594_1280.jpg"
    # Add more image URLs as needed
]

descriptions = {}

for url in image_urls:
  response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "What’s in this image?"},
          {
            "type": "image_url",
            "image_url": {
              "url": url,
            },
          },
        ],
      }
    ],
    max_tokens=300,
  )

  # print(response.choices[0])
  description = response.choices[0].message.content
  descriptions[url] = description

print('descriptions:', json.dumps(descriptions, indent=4))

NameError: name 'json' is not defined

# Creating a predefined set of attributes to extract from images

In [60]:
import json

'''
This code uses the OpenAI client object to make a chat completion request using the GPT-4 Turbo model.
It prompts the GPT to extract 7 pre-defined attributes from the description of the image input into 
the image-to-text response above. Then it converts the formatted string object into an object of attributes
that we can utilize further using json.
''' 

attributes = {}

for url, description in descriptions.items():
  response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", 
          # prompt to extract 7 key pre-defined attributes: color palette, subject matter, mood/emotion, style/aesthetic, setting/location, time period, cultural/symbolic representation
          "text": "You are a machine learning model trained to extract specific attributes from a detailed description of an image. Your task is to identify the following seven attributes from the given text and represent them as key-value pairs in a Python dictionary: 1. Color Palette: Identify the dominant colors or color schemes present in the image. 2. Subject Matter: Identify the main subject, object, or theme depicted in the image. 3. Mood/Emotion: Identify the overall mood, emotion, or feeling conveyed by the image. 4. Style/Aesthetic: Identify the artistic style, aesthetic, or visual characteristics of the image. 5. Setting/Location: Identify the physical setting, location, or environment depicted in the image. 6. Time Period: Identify the time period, era, or historical context represented in the image. 7. Cultural/Symbolic Representation: Identify any cultural, symbolic, or metaphorical representations present in the image. Please provide your output in the following format: {'Color Palette': 'one or two word adjectives', 'Subject Matter': 'one or two word adjectives', 'Mood/Emotion': 'one or two word adjectives', 'Style/Aesthetic': 'one or two word adjectives', 'Setting/Location': 'one or two word adjectives', 'Time Period': 'one or two word adjectives', 'Cultural/Symbolic Representation': 'one or two word adjectives'}; if an attribute is not obvious, infer from the context of the description an attribute, don't use 'none' or 'not specified' as an attribute. Format the output in double quotations so I can convert it into an object with json. The description is as follows:" + description,
  },
          {
            "type": "text",
            "text" : json.dumps(description)
          },
        ],
      }
    ],
    max_tokens=500,
  )

  # string reponse from prompt
  attributes_content = response.choices[0].message.content

  # object to work with data
  attributes_dict = json.loads(attributes_content)

  # append to attributes dictionary
  attributes[url] = attributes_dict

print('Image - Attributes Dictionary:\n', json.dumps(attributes, indent=4))

Image - Attributes Dictionary:
 {
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg": {
        "Color Palette": "vibrant green, vivid blue",
        "Subject Matter": "natural landscape",
        "Mood/Emotion": "peace, tranquility",
        "Style/Aesthetic": "picturesque",
        "Setting/Location": "nature reserve",
        "Time Period": "contemporary",
        "Cultural/Symbolic Representation": "environmental appreciation"
    },
    "https://cdn.pixabay.com/photo/2017/08/25/18/48/watercolor-2681039_1280.jpg": {
        "Color Palette": "vibrant, colorful",
        "Subject Matter": "abstract art",
        "Mood/Emotion": "dynamic, fluid",
        "Style/Aesthetic": "alcohol ink",
        "Setting/Location": "imaginary",
        "Time Period": "contemporary",
        "Cultural/Symbolic Representation": "organic"
    },
    "https://cdn.pixabay.com/photo/2017/03/12/1

## Rigorous Version of Pre-trained Model

In [18]:
def image_to_text(image_url):

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "Please provide a thorough, detailed, and artistic description of the image in the following URL: " + image_url,
            }
        ],
        max_tokens=300,
    )
    
    return response.choices[0].message.content

In [19]:
description = image_to_text("https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg")

print(description)

The image presents a serene and picturesque scene from a nature boardwalk in Madison, Wisconsin. The focus of the composition is an elegant, curved wooden walkway that invites viewers on a tranquil journey through a lush wetland environment. This gently winding pathway serves not only as a functional element allowing visitors to explore the natural beauty without disturbing the ecosystem but also as a visual guide leading the eye through the varied textures and shades of green of the landscape.

The setting is rich with an array of native plants, vivid in various greens that suggest a healthy, vibrant ecosystem. The foliage includes tall grasses which sway slightly, suggesting a gentle breeze. These grasses, interspersed with broader-leaved plants and punctuated occasionally by flowering blooms, create a tapestry of textures and shapes. The diversity of the plant life forms a dense, almost untamed layer that covers the ground and enriches the scene’s depth.

Overhead, a sky of mixed cl

In [21]:
def description_to_attributes(description):

    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a machine learning model trained to extract specific attributes from a detailed description of an image. Your task is to identify the following seven attributes from the given text and represent them as key-value pairs in a Python dictionary: 1. Color Palette: Identify the dominant colors or color schemes present in the image. 2. Subject Matter: Identify the main subject, object, or theme depicted in the image. 3. Mood/Emotion: Identify the overall mood, emotion, or feeling conveyed by the image. 4. Style/Aesthetic: Identify the artistic style, aesthetic, or visual characteristics of the image. 5. Setting/Location: Identify the physical setting, location, or environment depicted in the image. 6. Time Period: Identify the time period, era, or historical context represented in the image. 7. Cultural/Symbolic Representation: Identify any cultural, symbolic, or metaphorical representations present in the image. Please provide your output in the following format: {'Color Palette': 'one or two word adjectives', 'Subject Matter': 'one or two word adjectives', 'Mood/Emotion': 'one or two word adjectives', 'Style/Aesthetic': 'one or two word adjectives', 'Setting/Location': 'one or two word adjectives', 'Time Period': 'one or two word adjectives', 'Cultural/Symbolic Representation': 'one or two word adjectives'}; if an attribute is not obvious, infer from the context of the description an attribute, don't use 'none' or 'not specified' as an attribute. Format the output in double quotations so I can convert it into an object with json. The description is as follows:" + description,
            }
        ],
        max_tokens=300,
    )

    attributes_content = response.choices[0].message.content
    attributes_dict = json.loads(attributes_content)

    return attributes_dict

In [22]:
print(description_to_attributes(description))

{'Color Palette': 'vibrant greens, natural wood', 'Subject Matter': 'nature boardwalk', 'Mood/Emotion': 'serene, tranquil', 'Style/Aesthetic': 'picturesque, naturalistic', 'Setting/Location': 'lush wetland', 'Time Period': 'contemporary', 'Cultural/Symbolic Representation': 'harmony with nature'}


## Rigorous Version of Pre-Defined Model with specific set of labels

In [25]:
def description_to_attributes_predefined_labels(description):

    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a machine learning model trained to extract specific attributes from a detailed description of an image. Your task is to identify and select the most appropriate label from the following pre-defined set of attributes for each of the seven categories given below, and represent them as key-value pairs in a Python dictionary: 1. Color Palette: Select the dominant colors or color schemes present in the image from the following options: [Vibrant, Monochromatic, Pastel, Earthy, Neon, Muted, Bold, Subdued, Contrasting, Harmonious, Warm, Cool, Saturated, Desaturated, Primary, Secondary, Tertiary]. 2. Subject Matter: Choose the main subject, object, or theme depicted in the image from the following options: [Landscape, Portrait, Still life, Abstract, Urban, Nature, Architecture, Human figure, Animals, Plants, Buildings, Interiors, Vehicles, Food, Technology, Fantasy, Surreal, Mythological, Historical event, Everyday objects]. 3. Mood/Emotion: Pick the overall mood, emotion, or feeling conveyed by the image from the following options: [Serene, Energetic, Melancholic, Joyful, Mysterious, Peaceful, Dramatic, Whimsical, Intense, Reflective, Nostalgic, Romantic, Foreboding, Playful, Thoughtful, Soothing, Chaotic, Hopeful]. 4. Style/Aesthetic: Select the artistic style, aesthetic, or visual characteristics of the image from the following options: [Realism, Impressionism, Surrealism, Minimalism, Expressionism, Cubism, Romanticism, Baroque, Renaissance, Gothic, Modernism, Postmodernism, Abstract expressionism, Fauvism, Dadaism, Pop art, Conceptual art]. 5. Setting/Location: Choose the physical setting, location, or environment depicted in the image from the following options: [Cityscape, Countryside, Beach, Forest, Mountains, Urban, Rural, Suburban, Indoor, Outdoor, Coastal, Desert, Arctic, Tropical, Industrial, Historical, Futuristic, Mythical, Extraterrestrial]. 6. Time Period: Pick the time period, era, or historical context represented in the image from the following options: [Contemporary, Renaissance, Medieval, Modern, Ancient, Futuristic, Victorian, Industrial revolution, Middle Ages, Renaissance, Prehistoric, Classical, Baroque, Gothic, Romantic, Edwardian, Roaring twenties, Art Deco, Post-war]. 7. Cultural/Symbolic Representation: Select any cultural, symbolic, or metaphorical representations present in the image from the following options: [Religious, Political, Mythological, Historical, Indigenous, Pop culture, Folklore, Literary, Ethnic, National, International, Gender, Social class, Environmental, Technological, Symbolic, Ritualistic, Iconic, Allegorical, Abstract]. Please provide your output in the following format: {'Color Palette': 'selected option', 'Subject Matter': 'selected option', 'Mood/Emotion': 'selected option', 'Style/Aesthetic': 'selected option', 'Setting/Location': 'selected option', 'Time Period': 'selected option', 'Cultural/Symbolic Representation': 'selected option'}; if an attribute is not obvious, infer from the context of the description an attribute, don't use 'none' or 'not specified' as an attribute. Format the output in double quotations so I can convert it into an object with json. The description is as follows:" + description
            }
        ],
        max_tokens=300,
    )

    attributes_content = response.choices[0].message.content
    attributes_dict = json.loads(attributes_content)

    return attributes_dict

In [26]:
print(description_to_attributes_predefined_labels(description))

{'Color Palette': 'Vibrant', 'Subject Matter': 'Nature', 'Mood/Emotion': 'Serene', 'Style/Aesthetic': 'Realism', 'Setting/Location': 'Countryside', 'Time Period': 'Contemporary', 'Cultural/Symbolic Representation': 'Environmental'}


## Modification of Pre-defined Model with Specific Set of Attributes to provide the 2 best options

In [66]:
def description_to_attributes_predefined_two(description):

    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a machine learning model trained to extract specific attributes from a detailed description of an image. Your task is to identify and select the most appropriate label from the following pre-defined set of attributes for each of the seven categories given below, and represent them as key-value pairs in a Python dictionary: 1. Color Palette: Select up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Vibrant, Monochromatic, Pastel, Earthy, Neon, Muted, Bold]. 2. Subject Matter: Choose up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Landscape, Portrait, Still life, Abstract, Urban, Nature, Architecture]. 3. Mood/Emotion: Pick up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Serene, Energetic, Melancholic, Joyful, Mysterious, Peaceful, Dramatic]. 4. Style/Aesthetic: Select up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Realism, Impressionism, Surrealism, Minimalism, Expressionism, Cubism, Romanticism]. 5. Setting/Location: Choose up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Cityscape, Countryside, Beach, Forest, Mountains, Urban, Rural]. 6. Time Period: Pick up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Contemporary, Renaissance, Medieval, Modern, Ancient, Futuristic, Victorian]. 7. Cultural/Symbolic Representation: Select up to 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Religious, Political, Mythological, Historical, Indigenous, Pop culture, Folklore]. Please provide your output in the following format:  {'Color Palette': ['most associated label', ..., 'least associated label'],  'Subject Matter': ['most associated label', ..., 'least associated label'],  'Mood/Emotion': ['most associated label', ..., 'least associated label'], 'Style/Aesthetic': ['most associated label', ..., 'least associated label'],  'Setting/Location': ['most associated label', ..., 'least associated label'],  'Time Period': ['most associated label', ..., 'least associated label'],  'Cultural/Symbolic Representation': ['most associated label', ..., 'least associated label']}. If an attribute is not obvious, infer from the context of the description an attribute, don't use 'none' or 'not specified' as an attribute. Only select attribute values from the provided list of options. PLEASE DO NOT Assign any attribute that is not present in the pre-defined attributes. Format the output in double quotations so it can be converted into an object with json. The description is as follows:" + description
            }
        ],
        max_tokens=300,
    )

    attributes_content = response.choices[0].message.content
    attributes_dict = json.loads(attributes_content)

    return attributes_dict

In [67]:
print(description_to_attributes_predefined_two(description))

{'Color Palette': ['Earthy', 'Vibrant', 'Muted'], 'Subject Matter': ['Nature', 'Landscape'], 'Mood/Emotion': ['Serene', 'Peaceful'], 'Style/Aesthetic': ['Realism'], 'Setting/Location': ['Forest', 'Countryside'], 'Time Period': ['Contemporary'], 'Cultural/Symbolic Representation': ['Folklore']}


## Code to Quantify Comparison between two images to see which ones are closer

### Function to determine Images that are overall closer

In [108]:
def description_to_all_attributes(description):

    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a machine learning model trained to extract specific attributes from a detailed description of an image. Your task is to identify and select the most appropriate label from the following pre-defined set of attributes for each of the seven categories given below, and represent them as key-value pairs in a Python dictionary: 1. Color Palette: Select exaclty 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Vibrant, Monochromatic, Pastel, Earthy, Neon, Muted, Bold]. 2. Subject Matter: Select exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Landscape, Portrait, Still life, Abstract, Urban, Nature, Architecture]. 3. Mood/Emotion: Select exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Serene, Energetic, Melancholic, Joyful, Mysterious, Peaceful, Dramatic]. 4. Style/Aesthetic: Select exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Realism, Impressionism, Surrealism, Minimalism, Expressionism, Cubism, Romanticism]. 5. Setting/Location: Select exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Cityscape, Countryside, Beach, Forest, Mountains, Urban, Rural]. 6. Time Period: Select exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Contemporary, Renaissance, Medieval, Modern, Ancient, Futuristic, Victorian]. 7. Cultural/Symbolic Representation: Select Exactly 7 labels without repetition from only the following options, ordered from the most closely associated to the least associated with the image: [Religious, Political, Mythological, Historical, Indigenous, Pop culture, Folklore]. Please provide your output in the following format:  {'Color Palette': ['most associated label', ..., 'least associated label'],  'Subject Matter': ['most associated label', ..., 'least associated label'],  'Mood/Emotion': ['most associated label', ..., 'least associated label'], 'Style/Aesthetic': ['most associated label', ..., 'least associated label'],  'Setting/Location': ['most associated label', ..., 'least associated label'],  'Time Period': ['most associated label', ..., 'least associated label'],  'Cultural/Symbolic Representation': ['most associated label', ..., 'least associated label']}. If an attribute is not obvious, infer from the context of the description an attribute, don't use 'none' or 'not specified' as an attribute. Only select attribute values from the provided list of options. PLEASE DO NOT Assign any attribute that is not present in the pre-defined attributes. I repeat - no label in the result should be outside the predefined set of labels for each attribute. Format the output in double quotations so it can be converted into an object with json. The description is as follows:" + description
            }
        ],
        max_tokens=1000,
    )

    attributes_content = response.choices[0].message.content
    attributes_dict = json.loads(attributes_content)

    return attributes_dict

In [74]:
obj = description_to_all_attributes(description)
print(obj)

# print the length of the array for each attribute in the object

for key, value in obj.items():
    print(key, len(value))


{'Color Palette': ['Vibrant', 'Earthy', 'Muted', 'Pastel', 'Monochromatic', 'Bold', 'Neon'], 'Subject Matter': ['Nature', 'Landscape', 'Still life', 'Abstract', 'Urban', 'Portrait', 'Architecture'], 'Mood/Emotion': ['Peaceful', 'Serene', 'Joyful', 'Mysterious', 'Energetic', 'Melancholic', 'Dramatic'], 'Style/Aesthetic': ['Realism', 'Impressionism', 'Minimalism', 'Surrealism', 'Expressionism', 'Romanticism', 'Cubism'], 'Setting/Location': ['Forest', 'Countryside', 'Rural', 'Mountains', 'Beach', 'Urban', 'Cityscape'], 'Time Period': ['Contemporary', 'Modern', 'Futuristic', 'Medieval', 'Renaissance', 'Ancient', 'Victorian'], 'Cultural/Symbolic Representation': ['Indigenous', 'Historical', 'Folklore', 'Political', 'Mythological', 'Pop culture', 'Religious']}
Color Palette 7
Subject Matter 7
Mood/Emotion 7
Style/Aesthetic 7
Setting/Location 7
Time Period 7
Cultural/Symbolic Representation 7


## Implementing Manhattan Distance Based Method to quantify discimilarity between image labels

In [75]:
def array_ordering_discimilarity(arr1, arr2):

    index_dict = {element: i for i, element in enumerate(arr2)}

    # Calculate the sum of absolute distances
    total_distance = 0
    for i, element in enumerate(arr1):
        index_in_array2 = index_dict[element]
        distance = abs(i - index_in_array2)
        total_distance += distance

    return total_distance

In [76]:
array1 = ['a', 'b', 'c', 'd']
array2 = ['b', 'a', 'c', 'd']
similarity_score = array_ordering_discimilarity(array1, array2)
print("Similarity Score:", similarity_score)

Similarity Score: 2


In [77]:
test_obj = {
  "Color Palette": ["Vibrant", "Monochromatic", "Pastel", "Earthy", "Neon", "Muted", "Bold"],
  "Subject Matter": ["Landscape", "Portrait", "Still life", "Abstract", "Urban", "Nature", "Architecture"],
  "Mood/Emotion": ["Serene", "Energetic", "Melancholic", "Joyful", "Mysterious", "Peaceful", "Dramatic"],
  "Style/Aesthetic": ["Realism", "Impressionism", "Surrealism", "Minimalism", "Expressionism", "Cubism", "Romanticism"],
  "Setting/Location": ["Cityscape", "Countryside", "Beach", "Forest", "Mountains", "Urban", "Rural"],
  "Time Period": ["Contemporary", "Renaissance", "Medieval", "Modern", "Ancient", "Futuristic", "Victorian"],
  "Cultural/Symbolic Representation": ["Religious", "Political", "Mythological", "Historical", "Indigenous", "Pop culture", "Folklore"]
}

for key, value in test_obj.items():
    print(key, len(value))

Color Palette 7
Subject Matter 7
Mood/Emotion 7
Style/Aesthetic 7
Setting/Location 7
Time Period 7
Cultural/Symbolic Representation 7


In [78]:
def image_labels_discimilarity(image_attributes, user_attributes):

    total_distance = 0
    
    for key, value in image_attributes.items():

        user_value = user_attributes[key]
        distance = array_ordering_discimilarity(value, user_value)
        total_distance += distance
    
    return total_distance

In [79]:
test_distance = image_labels_discimilarity(test_obj, obj)
print("Test Distance:", test_distance)

Test Distance: 86


In [80]:
test_url1 = "https://upload.wikimedia.org/wikipedia/commons/e/e2/Lake_Tekapo_01.jpg"
test_url1_attributes = description_to_all_attributes(image_to_text(test_url1))
print(test_url1_attributes)

{'Color Palette': ['Vibrant', 'Earthy', 'Pastel', 'Muted', 'Monochromatic', 'Bold', 'Neon'], 'Subject Matter': ['Landscape', 'Nature', 'Still life', 'Abstract', 'Urban', 'Architecture', 'Portrait'], 'Mood/Emotion': ['Serene', 'Peaceful', 'Joyful', 'Mysterious', 'Dramatic', 'Energetic', 'Melancholic'], 'Style/Aesthetic': ['Realism', 'Impressionism', 'Romanticism', 'Minimalism', 'Expressionism', 'Surrealism', 'Cubism'], 'Setting/Location': ['Mountains', 'Countryside', 'Forest', 'Rural', 'Beach', 'Cityscape', 'Urban'], 'Time Period': ['Contemporary', 'Modern', 'Victorian', 'Renaissance', 'Medieval', 'Ancient', 'Futuristic'], 'Cultural/Symbolic Representation': ['Historical', 'Indigenous', 'Folklore', 'Mythological', 'Religious', 'Political', 'Pop culture']}


In [81]:
test_distance_url1 = image_labels_discimilarity(test_url1_attributes, test_obj)
print("Test Distance URL1:", test_distance_url1)

Test Distance URL1: 92


In [82]:
test_url2 = "https://upload.wikimedia.org/wikipedia/commons/8/82/Найкращі_миті_життя.jpg"
test_url2_attributes = description_to_all_attributes(image_to_text(test_url2))
print(test_url2_attributes)

{'Color Palette': ['Earthy', 'Muted', 'Vibrant', 'Pastel', 'Bold', 'Monochromatic', 'Neon'], 'Subject Matter': ['Nature', 'Portrait', 'Landscape', 'Still life', 'Abstract', 'Urban', 'Architecture'], 'Mood/Emotion': ['Serene', 'Peaceful', 'Joyful', 'Dramatic', 'Mysterious', 'Energetic', 'Melancholic'], 'Style/Aesthetic': ['Realism', 'Romanticism', 'Impressionism', 'Minimalism', 'Expressionism', 'Surrealism', 'Cubism'], 'Setting/Location': ['Countryside', 'Mountains', 'Forest', 'Rural', 'Beach', 'Cityscape', 'Urban'], 'Time Period': ['Contemporary', 'Modern', 'Victorian', 'Renaissance', 'Medieval', 'Ancient', 'Futuristic'], 'Cultural/Symbolic Representation': ['Folklore', 'Historical', 'Mythological', 'Indigenous', 'Religious', 'Political', 'Pop culture']}


In [83]:
test_distance_url2 = image_labels_discimilarity(test_url2_attributes, test_obj)
print("Test Distance URL2:", test_distance_url2)

Test Distance URL2: 100


In [85]:
test_url3 = "https://upload.wikimedia.org/wikipedia/commons/6/69/Russia._Industry_Orekhovo-Zuevo._img-004.jpg"
test_url3_attributes = description_to_all_attributes(image_to_text(test_url3))
print(test_url3_attributes)

{'Color Palette': ['Monochromatic', 'Muted', 'Earthy', 'Vibrant', 'Bold', 'Pastel', 'Neon'], 'Subject Matter': ['Architecture', 'Urban', 'Landscape', 'Portrait', 'Still life', 'Abstract', 'Nature'], 'Mood/Emotion': ['Melancholic', 'Serene', 'Dramatic', 'Peaceful', 'Mysterious', 'Energetic', 'Joyful'], 'Style/Aesthetic': ['Realism', 'Minimalism', 'Expressionism', 'Surrealism', 'Romanticism', 'Impressionism', 'Cubism'], 'Setting/Location': ['Urban', 'Rural', 'Cityscape', 'Countryside', 'Forest', 'Mountains', 'Beach'], 'Time Period': ['Modern', 'Contemporary', 'Victorian', 'Medieval', 'Renaissance', 'Futuristic', 'Ancient'], 'Cultural/Symbolic Representation': ['Historical', 'Political', 'Indigenous', 'Folklore', 'Religious', 'Pop culture', 'Mythological']}


In [86]:
test_distance_url3 = image_labels_discimilarity(test_url3_attributes, test_obj)
print("Test Distance URL3:", test_distance_url3)

Test Distance URL3: 112


In [87]:
dis_12 = image_labels_discimilarity(test_url1_attributes, test_url2_attributes)
dis_13 = image_labels_discimilarity(test_url1_attributes, test_url3_attributes)
dis_23 = image_labels_discimilarity(test_url2_attributes, test_url3_attributes)

print("Disimilarity between test_url1 and test_url2:", dis_12)
print("Disimilarity between test_url1 and test_url3:", dis_13)
print("Disimilarity between test_url2 and test_url3:", dis_23)

Disimilarity between test_url1 and test_url2: 32
Disimilarity between test_url1 and test_url3: 102
Disimilarity between test_url2 and test_url3: 98


## Putting the above code into a function that does it all

- Takes various images and calculates and plots them on a graph based on their dissimilairty distance.
- Is able to take in uploaded images - not just wiki media URL - specifically the art images from our dataset.
- It is able to calculate total dissimilarity and specific label discimilairty between images
- more trouble-shooting so that the image-to-text to json works every time.

### Function to calculate dissimilarity for a specific label

In [88]:
def label_dissimilarity(image_attributes, user_attributes, label):
    
    image_value = image_attributes.get(label)
    user_value = user_attributes.get(label)
    
    
    # Calculate the dissimilarity score between the two arrays
    distance = array_ordering_discimilarity(image_value, user_value)
    
    return distance


In [89]:
dis_12_color = label_dissimilarity(test_url1_attributes, test_url2_attributes, "Color Palette")
dis_12_subject = label_dissimilarity(test_url1_attributes, test_url2_attributes, "Subject Matter")

print("Disimilarity between test_url1 and test_url2 for Color Palette:", dis_12_color)
print("Disimilarity between test_url1 and test_url2 for Subject Matter:", dis_12_subject)

Disimilarity between test_url1 and test_url2 for Color Palette: 8
Disimilarity between test_url1 and test_url2 for Subject Matter: 12


### Enhanced Image dissimilarity functino with least dissimilar label

In [92]:
def image_labels_discimilarity_mod(image_attributes, user_attributes):
    
    distances = {}
    total_distance = 0
    
    for key, value in image_attributes.items():
        user_value = user_attributes.get(key)
        
        # If the user attribute is missing, skip
        if user_value is None:
            continue
        
        distance = array_ordering_discimilarity(value, user_value)
        total_distance += distance
        distances[key] = distance
    
    # Sort labels based on distances in ascending order
    ordered_labels = sorted(distances, key=distances.get)
    
    return total_distance, ordered_labels

In [93]:
dis_12_mod, label_12 = image_labels_discimilarity_mod(test_url1_attributes, test_url2_attributes)

print("Disimilarity between test_url1 and test_url2 (Modified):", dis_12_mod)
print("Ordered Labels:", label_12)

Disimilarity between test_url1 and test_url2 (Modified): 32
Ordered Labels: ['Time Period', 'Mood/Emotion', 'Style/Aesthetic', 'Setting/Location', 'Cultural/Symbolic Representation', 'Color Palette', 'Subject Matter']


### Look into other metrics to quantify similairty between image labels
### Potentially enhance attribute list for images

In [95]:
art_url_1 = "https://upload.wikimedia.org/wikipedia/commons/0/0d/Great_Wave_off_Kanagawa2.jpg"
art_url_2 = "https://upload.wikimedia.org/wikipedia/commons/9/94/The_Nightwatch_by_Rembrandt_-_Rijksmuseum.jpg"
art_url_3 = "https://upload.wikimedia.org/wikipedia/commons/b/b6/Sebastiano_Ricci_002.jpg"

In [96]:
text_1 = image_to_text(art_url_1)
attributes_1 = description_to_all_attributes(text_1)

print("Attributes for Art URL 1:", attributes_1)

Attributes for Art URL 1: {'Color Palette': ['Vibrant', 'Muted', 'Bold', 'Monochromatic', 'Earthy', 'Pastel', 'Neon'], 'Subject Matter': ['Nature', 'Abstract', 'Portrait', 'Landscape', 'Still life', 'Urban', 'Architecture'], 'Mood/Emotion': ['Dramatic', 'Mysterious', 'Serene', 'Energetic', 'Melancholic', 'Peaceful', 'Joyful'], 'Style/Aesthetic': ['Realism', 'Expressionism', 'Surrealism', 'Impressionism', 'Romanticism', 'Minimalism', 'Cubism'], 'Setting/Location': ['Beach', 'Mountains', 'Countryside', 'Forest', 'Rural', 'Cityscape', 'Urban'], 'Time Period': ['Ancient', 'Modern', 'Contemporary', 'Medieval', 'Renaissance', 'Victorian', 'Futuristic'], 'Cultural/Symbolic Representation': ['Historical', 'Mythological', 'Folklore', 'Indigenous', 'Religious', 'Political', 'Pop culture']}


In [102]:
# testing against test object

dis_1 = image_labels_discimilarity(attributes_1, test_obj)
print("Disimilarity between Art URL 1 and test object:", dis_1)

Disimilarity between Art URL 1 and test object: 104


In [109]:
text_2 = image_to_text(art_url_2)
attributes_2 = description_to_all_attributes(text_2)

print("Attributes for Art URL 2:", attributes_2)

Attributes for Art URL 2: {'Color Palette': ['Bold', 'Muted', 'Vibrant', 'Monochromatic', 'Earthy', 'Pastel', 'Neon'], 'Subject Matter': ['Portrait', 'Urban', 'Abstract', 'Still life', 'Landscape', 'Nature', 'Architecture'], 'Mood/Emotion': ['Dramatic', 'Mysterious', 'Energetic', 'Serene', 'Melancholic', 'Peaceful', 'Joyful'], 'Style/Aesthetic': ['Realism', 'Romanticism', 'Expressionism', 'Impressionism', 'Surrealism', 'Cubism', 'Minimalism'], 'Setting/Location': ['Urban', 'Cityscape', 'Countryside', 'Beach', 'Forest', 'Mountains', 'Rural'], 'Time Period': ['Renaissance', 'Victorian', 'Medieval', 'Modern', 'Contemporary', 'Ancient', 'Futuristic'], 'Cultural/Symbolic Representation': ['Historical', 'Political', 'Religious', 'Folklore', 'Mythological', 'Indigenous', 'Pop culture']}


In [110]:
# testing against test object

dis_2 = image_labels_discimilarity(attributes_2, test_obj)
print("Disimilarity between Art URL 2 and test object:", dis_2)

Disimilarity between Art URL 2 and test object: 96


In [111]:
text_3 = image_to_text(art_url_3)
attributes_3 = description_to_all_attributes(text_3)

print("Attributes for Art URL 3:", attributes_3)

Attributes for Art URL 3: {'Color Palette': ['Muted', 'Earthy', 'Pastel', 'Vibrant', 'Monochromatic', 'Bold', 'Neon'], 'Subject Matter': ['Portrait', 'Still life', 'Architecture', 'Nature', 'Abstract', 'Urban', 'Landscape'], 'Mood/Emotion': ['Serene', 'Peaceful', 'Dramatic', 'Mysterious', 'Melancholic', 'Joyful', 'Energetic'], 'Style/Aesthetic': ['Realism', 'Romanticism', 'Impressionism', 'Expressionism', 'Surrealism', 'Minimalism', 'Cubism'], 'Setting/Location': ['Rural', 'Countryside', 'Cityscape', 'Urban', 'Forest', 'Mountains', 'Beach'], 'Time Period': ['Renaissance', 'Victorian', 'Modern', 'Medieval', 'Contemporary', 'Ancient', 'Futuristic'], 'Cultural/Symbolic Representation': ['Religious', 'Historical', 'Mythological', 'Folklore', 'Political', 'Indigenous', 'Pop culture']}


In [112]:
# testing against test object

dis_3 = image_labels_discimilarity(attributes_3, test_obj)
print("Disimilarity between Art URL 3 and test object:", dis_3)

Disimilarity between Art URL 3 and test object: 102


In [115]:
dis_art1_art2 = image_labels_discimilarity(attributes_1, attributes_2)
dis_art1_art3 = image_labels_discimilarity(attributes_1, attributes_3)
dis_art2_art3 = image_labels_discimilarity(attributes_2, attributes_3)

print("Disimilarity between Art URL 1 and Art URL 2:", dis_art1_art2)
print("Disimilarity between Art URL 1 and Art URL 3:", dis_art1_art3)
print("Disimilarity between Art URL 2 and Art URL 3:", dis_art2_art3)

dis_art1_art2_mod, label_art1_art2 = image_labels_discimilarity_mod(attributes_1, attributes_2)

print("Disimilarity between Art URL 1 and Art URL 2 (Modified):", dis_art1_art2_mod)
print("Ordered Labels:", label_art1_art2)

dis_art2_art3_mod, label_art2_art3 = image_labels_discimilarity_mod(attributes_2, attributes_3)

print("Disimilarity between Art URL 2 and Art URL 3 (Modified):", dis_art2_art3_mod)
print("Ordered Labels:", label_art2_art3)

Disimilarity between Art URL 1 and Art URL 2: 78
Disimilarity between Art URL 1 and Art URL 3: 102
Disimilarity between Art URL 2 and Art URL 3: 74
Disimilarity between Art URL 1 and Art URL 2 (Modified): 78
Ordered Labels: ['Mood/Emotion', 'Color Palette', 'Style/Aesthetic', 'Cultural/Symbolic Representation', 'Subject Matter', 'Time Period', 'Setting/Location']
Disimilarity between Art URL 2 and Art URL 3 (Modified): 74
Ordered Labels: ['Time Period', 'Style/Aesthetic', 'Cultural/Symbolic Representation', 'Color Palette', 'Setting/Location', 'Subject Matter', 'Mood/Emotion']


# VERY IMPORTANT: ENSURING THAT THE LABEL ORDERING FOR EACH IMAGE ONLY CONTAINS LABELS FROM PRE_DEFINED SET OF LABELS. RUN ABOVE CODE CELL TO UNDERSATND MORE ABOUT THIS ERROR.